In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
from pyts.image import GramianAngularField
from pyts.datasets import load_gunpoint
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.utils import resample
from imblearn.combine import SMOTETomek
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import TomekLinks, NearMiss
from sklearn.model_selection import train_test_split
import h5py
from PIL import Image
from torchvision import datasets, transforms

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision

import torch.nn as nn
from torchvision.models import alexnet, vgg16, resnet152, resnet18, vgg19
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import classification_report, recall_score, f1_score, precision_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import pairwise_distances

In [3]:
df_hb_normal = pd.read_csv('ptbdb_normal.csv', header=None)
df_hb_abnormal = pd.read_csv('ptbdb_abnormal.csv', header=None)

In [4]:
df_hb = pd.concat([df_hb_normal, df_hb_abnormal])

In [5]:
df_hb = df_hb.reset_index().drop(['index'], axis=1)

In [6]:
x = df_hb.iloc[:,:187]
y = df_hb[187]

In [7]:
x_train, x_test, y_train, y_test = train_test_split(x, y, stratify=y, test_size=0.2, random_state=42)

In [8]:
gasf = GramianAngularField(image_size=150, method='difference')
x_gasf_train = gasf.fit_transform(x_train)
x_gasf_test = gasf.transform(x_test)

In [12]:
with h5py.File('hb_data_ptb.hdf5', mode='w') as hdf5_file:
    hdf5_file.create_dataset("x_train", (len(x_gasf_train), 224, 224), np.float32)
    hdf5_file.create_dataset("y_train", (len(y_train),), np.int32)
    hdf5_file.create_dataset("x_test", (len(x_gasf_test), 224, 224), np.float32)
    hdf5_file.create_dataset("y_test", (len(y_test),), np.int32)
    
    transform = transforms.Compose([transforms.Resize((224, 224))])
    
    for i, (x, y) in enumerate(zip(x_gasf_train, y_train)):
        image = Image.fromarray(x.astype(float))
        t = np.array(transform(image))
        hdf5_file["x_train"][i, ...] = t
        hdf5_file["y_train"][i] = y

    for i, (x, y) in enumerate(zip(x_gasf_test, y_test)):
        image = Image.fromarray(x.astype(float))
        t = np.array(transform(image))
        hdf5_file["x_test"][i, ...] = t
        hdf5_file["y_test"][i] = y

In [12]:
def recurrence_plot(s, eps=None, steps=None):
    result = []
    if eps==None: eps=0.1
    if steps==None: steps=10
    d = pairwise_distances(s[:, None])
    d = d / eps
    d[d > steps] = steps
    return d/5. - 1

In [25]:
with h5py.File('hb_data_ptb_rp.hdf5', mode='w') as hdf5_file:
    hdf5_file.create_dataset("x_train", (len(x_train), 224, 224), np.float32)
    hdf5_file.create_dataset("y_train", (len(y_train),), np.int32)
    hdf5_file.create_dataset("x_test", (len(x_test), 224, 224), np.float32)
    hdf5_file.create_dataset("y_test", (len(y_test),), np.int32)
    
    transform = transforms.Compose([transforms.Resize((224, 224))])
    
    for i, (x, y) in enumerate(zip(x_train.values, y_train)):
        r = recurrence_plot(x, steps=10)
        image = Image.fromarray(r.astype(float))
        t = np.array(transform(image))
        hdf5_file["x_train"][i, ...] = t
        hdf5_file["y_train"][i] = y

    for i, (x, y) in enumerate(zip(x_test.values, y_test)):
        r = recurrence_plot(x, steps=10)
        image = Image.fromarray(r.astype(float))
        t = np.array(transform(image))
        hdf5_file["x_test"][i, ...] = t
        hdf5_file["y_test"][i] = y

In [36]:
with h5py.File('hb_data_ptb_mixed.hdf5', mode='w') as hdf5_file:
    hdf5_file.create_dataset("x_train", (len(x_train), 3, 224, 224), np.float32)
    hdf5_file.create_dataset("y_train", (len(y_train),), np.int32)
    hdf5_file.create_dataset("x_test", (len(x_test), 3, 224, 224), np.float32)
    hdf5_file.create_dataset("y_test", (len(y_test),), np.int32)
    
    transform = transforms.Compose([transforms.Resize((224, 224))])
    
    for i, (x1, x2, y) in enumerate(zip(x_gasf_train, x_train.values, y_train)):
        image = Image.fromarray(x1.astype(float))
        t = np.array(transform(image))
        r = recurrence_plot(x2, steps=10)
        image = Image.fromarray(r.astype(float))
        t2 = np.array(transform(image))
        hdf5_file["x_train"][i, 0, ...] = t
        hdf5_file["x_train"][i, 1, ...] = t2
        hdf5_file["x_train"][i, 2, ...] = (t + t2) / 2
        hdf5_file["y_train"][i] = y

    for i, (x1, x2, y) in enumerate(zip(x_gasf_test, x_test.values, y_test)):
        image = Image.fromarray(x1.astype(float))
        t = np.array(transform(image))
        r = recurrence_plot(x2, steps=10)
        image = Image.fromarray(r.astype(float))
        t2 = np.array(transform(image))
        hdf5_file["x_test"][i, 0, ...] = t
        hdf5_file["x_test"][i, 1, ...] = t2
        hdf5_file["x_test"][i, 2, ...] = (t + t2) / 2
        hdf5_file["y_test"][i] = y

In [46]:
# loading checkpoint from MIT BIH model
ckpt_recall = torch.load('res_net_test_recall_best_cmb.chk')
res_net_saved_recall = resnet18(pretrained=True)
for param in res_net_saved_recall.parameters():
    param.requires_grad = False
    
# MIT BIH model had 2 frozen convolutional layers, 3 trainable FC layers
# In this model all convolutional layers are frozen but 3 FCs are trainable
# The idea is that the feature generator from MIT BIH will be frozen
num_ftrs = res_net_saved_recall.fc.in_features
res_net_saved_recall.fc = nn.Sequential(
                nn.Linear(in_features=num_ftrs, out_features=256, bias=False),
                nn.ReLU(),
                nn.Linear(in_features=256, out_features=128, bias=True),
                nn.ReLU(),
                nn.Linear(in_features=128, out_features=5, bias=True))
res_net_saved_recall.load_state_dict(ckpt_recall['net'])
res_net_saved_recall.fc = nn.Sequential(
                nn.Linear(in_features=num_ftrs, out_features=256, bias=False),
                nn.ReLU(),
                nn.Linear(in_features=256, out_features=128, bias=True),
                nn.ReLU(),
                nn.Linear(in_features=128, out_features=2, bias=True))
res_net_saved_recall.cuda()   

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co

In [18]:
# Function for moving tensor or model to GPU
def cuda(xs):
    if torch.cuda.is_available():
        if not isinstance(xs, (list, tuple)):
            return xs.cuda()
        else:
            return [x.cuda() for x in xs]
    else:
        return xs

# Custom class for defining dataset for training with augmentation
class Dataset_Hdf5(Dataset):

    def __init__(self, path, data_type):
        """ Intialize the dataset
        """
        self.path = path
        self.file = h5py.File(path, 'r')
        self.images = self.file['x_{}'.format(data_type)]
        self.labels = self.file['y_{}'.format(data_type)]
                
        self.len = self.images.shape[0]
        if data_type == 'train':
            self.transform = transforms.Compose([transforms.ToTensor()])
        else:
            self.transform = transforms.Compose([transforms.ToTensor()])

    # You must override __getitem__ and __len__
    def __getitem__(self, index):
        """ Get a sample from the dataset
        """
        # unsqueeze adds dimension to image -> converts to 1x224x224 since we don't have rgb
        return self.transform(self.images[index].astype('float32')), \
                torch.tensor(self.labels[index], dtype=torch.long)

    def __len__(self):
        """
        Total number of samples in the dataset
        """
        return self.len
    
# Custom class for defining dataset for training with augmentation
class Dataset_Hdf5_3C(Dataset):

    def __init__(self, path, data_type):
        """ Intialize the dataset
        """
        self.path = path
        self.file = h5py.File(path, 'r')
        self.images = self.file['x_{}'.format(data_type)]
        self.labels = self.file['y_{}'.format(data_type)]
                
        self.len = self.images.shape[0]

    # You must override __getitem__ and __len__
    def __getitem__(self, index):
        """ Get a sample from the dataset
        """
        # unsqueeze adds dimension to image -> converts to 1x224x224 since we don't have rgb
        return torch.tensor(self.images[index].astype('float32')), \
                torch.tensor(self.labels[index], dtype=torch.long)

    def __len__(self):
        """
        Total number of samples in the dataset
        """
        return self.len    

In [39]:
hb_train_loader = torch.utils.data.DataLoader(Dataset_Hdf5('/home/asif/heartbeat/hb_data_ptb_rp.hdf5', 'train'), 
                                                batch_size=64, shuffle=True)
hb_test_loader = torch.utils.data.DataLoader(Dataset_Hdf5('/home/asif/heartbeat/hb_data_ptb_rp.hdf5', 'test'), 
                                                batch_size=64, shuffle=False)

In [47]:
hb_train_loader2 = torch.utils.data.DataLoader(Dataset_Hdf5_3C('/home/asif/heartbeat/hb_data_ptb_mixed.hdf5', 'train'), 
                                                batch_size=64, shuffle=True)
hb_test_loader2 = torch.utils.data.DataLoader(Dataset_Hdf5_3C('/home/asif/heartbeat/hb_data_ptb_mixed.hdf5', 'test'), 
                                                batch_size=64, shuffle=False)

In [20]:
class_weights = cuda(torch.tensor([2.0, 1.0]))
criterion = nn.CrossEntropyLoss(weight=class_weights)

In [21]:
optimizer_res_net = torch.optim.Adam([
                                          {"params": res_net_saved_recall.fc[0].parameters(), "lr": 0.001},
                                          {"params": res_net_saved_recall.fc[2].parameters(), "lr": 0.001},
                                          {"params": res_net_saved_recall.fc[4].parameters(), "lr": 0.001},
                                           ],  
                                lr=0.0001, betas=(0.9, 0.999))

In [22]:
def train(net, train_loader, criterion, optimizer, test_loader, num_epochs=30):
    net.train()
    train_acc_max = 0
    test_acc_max = 0

    scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma = 0.95)
    
    for epoch in range(num_epochs):  # loop over the dataset multiple times
        net.train()

        total = 0
        correct = 0
   
        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = cuda(data)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            # outputs = net(inputs.expand(-1, 3, -1, -1))
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
        
        scheduler.step()
        
        print('End of epoch {}, Loss {}'.format(epoch + 1, running_loss / len(train_loader)))
        
        train_acc = correct / total
        print('Train accuracy: {}'.format(train_acc))
        test_acc, all_true, all_pred = test(net, test_loader)
        print('Test accuracy: {}'.format(test_acc))
        precision = precision_score(all_true, all_pred, average='macro')
        recall = recall_score(all_true, all_pred, average='macro')
        f1 = f1_score(all_true, all_pred, average='macro')
        print('Test precision: {}'.format(precision))
        print('Test recall: {}'.format(recall))
        print('Test f1: {}'.format(f1))

    print('Finished Training')
    
def test(net, test_loader):
    net.eval()
    correct = 0
    total = 0
    all_true = []
    all_pred = []
    with torch.no_grad():
        for i, data in enumerate(test_loader, 0):
            images, labels = cuda(data)
            all_true.extend(labels.cpu().tolist())
#             outputs = net(images.expand(-1, 3, -1, -1))
            outputs = net(images)
            _, predicted = torch.max(outputs, 1)
            all_pred.extend(predicted.cpu().tolist())
#             predicted = predicted.float()
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    acc = correct / total
#     print('Accuracy of the network on the images: %d %%' % (100 * acc))
    return acc, all_true, all_pred

In [51]:
train(res_net_saved_recall, hb_train_loader2, criterion, optimizer_res_net, hb_test_loader2, 30)

End of epoch 1, Loss 0.5297282012281838
Train accuracy: 0.7288033674082982
Test accuracy: 0.7475094469254552
Test precision: 0.7328944646659404
Test recall: 0.7886693936788673
Test f1: 0.7295574334876124
End of epoch 2, Loss 0.45415015575977474
Train accuracy: 0.776393780603041
Test accuracy: 0.8110614908965991
Test precision: 0.7666388169070044
Test recall: 0.7494160602828079
Test f1: 0.757004015303564
End of epoch 3, Loss 0.4117244554089976
Train accuracy: 0.810239670131432
Test accuracy: 0.8392305049811062
Test precision: 0.7992057175402435
Test recall: 0.8373533829103837
Test f1: 0.812732993021481
End of epoch 4, Loss 0.37393098725722385
Train accuracy: 0.8356670389141826
Test accuracy: 0.8570937822054276
Test precision: 0.8183661723046516
Test recall: 0.8493423768522297
Test f1: 0.8307207998076526
End of epoch 5, Loss 0.36926299243510424
Train accuracy: 0.8319731981788506
Test accuracy: 0.8461009962212298
Test precision: 0.8258977983109367
Test recall: 0.7725381324984505
Test f1: 

## MTF

In [9]:
# loading checkpoint from MIT BIH model
ckpt_recall_mtf = torch.load('res_net_mtf_gaf_rp_test_recall_best.chk')
res_net_saved_recall = resnet18(pretrained=True)
for param in res_net_saved_recall.parameters():
    param.requires_grad = False
    
# MIT BIH model had 2 frozen convolutional layers, 3 trainable FC layers
# In this model all convolutional layers are frozen but 3 FCs are trainable
# The idea is that the feature generator from MIT BIH will be frozen
num_ftrs = res_net_saved_recall.fc.in_features
res_net_saved_recall.fc = nn.Sequential(
                nn.Linear(in_features=num_ftrs, out_features=256, bias=False),
                nn.ReLU(),
                nn.Linear(in_features=256, out_features=128, bias=True),
                nn.ReLU(),
                nn.Linear(in_features=128, out_features=5, bias=True))
res_net_saved_recall.load_state_dict(ckpt_recall_mtf['net'])
res_net_saved_recall.fc = nn.Sequential(
                nn.Linear(in_features=num_ftrs, out_features=256, bias=False),
                nn.ReLU(),
                nn.Linear(in_features=256, out_features=128, bias=True),
                nn.ReLU(),
                nn.Linear(in_features=128, out_features=2, bias=True))
res_net_saved_recall.cuda()   

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [10]:
def get_quantiles(min_value=0, max_val=1, k=10):
    c = (max_val - min_value)/k
    b = min_value + c
    d = []
    for i in range(1, k):
        d.append(b)
        b += c
    d.append(max_val)
    return d

quantiles = get_quantiles()

def value_to_quantile(x):
    for i, k in enumerate(quantiles):
        if x <= k:
            return i
    return 0

def get_mtf(x, size=10):
    q = np.vectorize(value_to_quantile)(x)
    r = np.zeros((q.shape[0], q.shape[0]))
    y = np.zeros((size, size))
    for i in range(x.shape[0] - 1):
        y[q[i], q[i + 1]] += 1
    y = y / y.sum(axis=1, keepdims=True)
    y[np.isnan(y)] = 0
    
    for i in range(r.shape[0]):
        for j in range(r.shape[1]):
            r[i, j] = y[q[i], q[j]]
    return r / 5. - 1

In [14]:
np.seterr(divide='ignore', invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [16]:
# 3 channels, this time with GAF, RP and MTF
with h5py.File('/home/asif/heartbeat/hb_data_ptb_balanced_gaf_rp_mtf.hdf5', mode='w') as hdf5_file:
    hdf5_file.create_dataset("x_train", (len(x_gasf_train), 3, 224, 224), np.float32)
    hdf5_file.create_dataset("y_train", (len(y_train),), np.int32)
    hdf5_file.create_dataset("x_test", (len(x_gasf_test), 3, 224, 224), np.float32)
    hdf5_file.create_dataset("y_test", (len(y_test),), np.int32)
    
    transform = transforms.Compose([transforms.Resize((224, 224))])
    
    for i, (x1, x2, y) in enumerate(zip(x_gasf_train, x_train.values, y_train)):
        image = Image.fromarray(x1.astype(float))
        t = np.array(transform(image))
        r = recurrence_plot(x2, steps=10)
        mtf = get_mtf(x2)
        image = Image.fromarray(r.astype(float))
        t2 = np.array(transform(image))
        image = Image.fromarray(mtf.astype(float))
        t3 = np.array(transform(image))
        hdf5_file["x_train"][i, 0, ...] = t
        hdf5_file["x_train"][i, 1, ...] = t2
        hdf5_file["x_train"][i, 2, ...] = t3
        hdf5_file["y_train"][i] = y

    for i, (x1, x2, y) in enumerate(zip(x_gasf_test, x_test.values, y_test)):
        image = Image.fromarray(x1.astype(float))
        t = np.array(transform(image))
        r = recurrence_plot(x2, steps=10)
        mtf = get_mtf(x2)
        image = Image.fromarray(r.astype(float))
        t2 = np.array(transform(image))
        image = Image.fromarray(mtf.astype(float))
        t3 = np.array(transform(image))
        hdf5_file["x_test"][i, 0, ...] = t
        hdf5_file["x_test"][i, 1, ...] = t2
        hdf5_file["x_test"][i, 2, ...] = t3
        hdf5_file["y_test"][i] = y

In [24]:
hb_train_loader3 = torch.utils.data.DataLoader(Dataset_Hdf5_3C('/home/asif/heartbeat/hb_data_ptb_balanced_gaf_rp_mtf.hdf5', 'train'), 
                                                batch_size=64, shuffle=True)
hb_test_loader3 = torch.utils.data.DataLoader(Dataset_Hdf5_3C('/home/asif/heartbeat/hb_data_ptb_balanced_gaf_rp_mtf.hdf5', 'test'), 
                                                batch_size=64, shuffle=False)

In [25]:
train(res_net_saved_recall, hb_train_loader3, criterion, optimizer_res_net, hb_test_loader3, 30)

End of epoch 1, Loss 0.36358735792256974
Train accuracy: 0.8420238811098703
Test accuracy: 0.8962555822741326
Test precision: 0.8628725938324857
Test recall: 0.8977493916559542
Test f1: 0.8770307394476523
End of epoch 2, Loss 0.2538518090169508
Train accuracy: 0.899836783781462
Test accuracy: 0.9203023016145654
Test precision: 0.890689838786677
Test recall: 0.9254253703871409
Test f1: 0.9051651989678364
End of epoch 3, Loss 0.19198271791849816
Train accuracy: 0.9252641525642127
Test accuracy: 0.947440742013054
Test precision: 0.9312358490674961
Test recall: 0.939274679832851
Test f1: 0.9351330166177378
End of epoch 4, Loss 0.16522728530997102
Train accuracy: 0.9367751911347822
Test accuracy: 0.9440054963929921
Test precision: 0.9235311721824763
Test recall: 0.9410779538940488
Test f1: 0.9316905225500182
End of epoch 5, Loss 0.13975154940079856
Train accuracy: 0.944506485697105
Test accuracy: 0.9367914805908623
Test precision: 0.9102379445488396
Test recall: 0.9436862767697842
Test f1: 